In [1]:
import numpy as np
import cv2

image=cv2.imread('digits.png')
gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
small=cv2.pyrDown(image)  #Reducing Image size
cv2.imshow('Digits Image',small)
cv2.waitKey(0)
cv2.destroyAllWindows()

cells=[np.hsplit(row,100) for row in np.split(gray,50)] 
x=np.array(cells)
print('The shape of our cells array : '+str(x.shape))

# Now we prepare train_data and test_data.
train = x[:,:50].reshape(-1,400).astype(np.float32) # Size = (2500,400)
test = x[:,50:100].reshape(-1,400).astype(np.float32) # Size = (2500,400)
# Create labels for train and test data

k = np.arange(10)
train_labels = np.repeat(k,250)[:,np.newaxis]
test_labels = train_labels.copy()
# Initiate kNN, train the data, then test it with test data for k=1
knn = cv2.ml.KNearest_create()
knn.train(train, cv2.ml.ROW_SAMPLE, train_labels)
ret,result,neighbours,dist = knn.findNearest(test,k=5)

# Now we check the accuracy of classification
# For that, compare the result with test_labels and check which are wrong
matches = result==test_labels
correct = np.count_nonzero(matches)
accuracy = correct*100.0/result.size

print("Accuracy is = %.2f" %accuracy +"%")



The shape of our cells array : (50L, 100L, 20L, 20L)
Accuracy is = 91.76%


In [13]:
import cv2
import numpy as np

def x_cord_contour(contour):
    #this fun take a counter from find contour it then outputs the x.centroid co-ordinate
    if cv2.contourArea(contour)>10:
        M = cv2.moments(contour)
        return(int(M['m10']/M['m00']))
    
def makeSquare(not_square):
    BLACK=[0,0,0]
    img_dim=not_square.shape
    height=img_dim[0]
    width=img_dim[1]
    
    if(height==width):
        square=not_square
        return square
    else:
        doublesize=cv2.resize(not_square,(2*width,2*height),interpolation=cv2.INTER_CUBIC)
        height=height*2;
        width=width*2;
        
        if(height > width ):
            pad=(height-width)/2;
            
            doublesize_square=cv2.copyMakeBorder(doublesize,0,0,pad,pad,cv2.BORDER_CONSTANT,value=BLACK)
       
        else:
            pad=(width-height)/2;
            doublesize_square=cv2.copyMakeBorder(doublesize,pad,pad,0,0,cv2.BORDER_CONSTANT,value=BLACK)
            
    doublesize_square_dim=doublesize_square.shape
    
    return doublesize_square

def resize_to_pixels(dimensions,image):
    buffer_fix=4
    dimensions=dimensions-buffer_fix
    squared=image
    r=float(dimensions)/squared.shape[1]
    dim=(dimensions,int(squared.shape[0]*r))
    resized=cv2.resize(image,dim,interpolation=cv2.INTER_AREA)
    img_dim2=resized.shape
    height_r=img_dim2[0]
    width_r=img_dim2[1]
    BLACK=[0,0,0]
    if(height_r > width_r):
        resized=cv2.copyMakeBorder(resized,0,0,0,1,cv2.BORDER_CONSTANT,value=BLACK)
    if(height_r < width_r):
        resized=cv2.copyMakeBorder(resized,1,0,0,0,cv2.BORDER_CONSTANT,value=BLACK)
    p=2
    
    Resizedimg=cv2.copyMakeBorder(resized,p,p,p,p,cv2.BORDER_CONSTANT,value=BLACK)
    img_dim=Resizedimg.shape
    height=img_dim[0]
    width=img_dim[1]
        
    return Resizedimg


In [29]:
import cv2
import numpy as np

image=cv2.imread('8.png')
cv2.imshow('Digits Image',image)
cv2.waitKey(0)


gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
gray = cv2.bilateralFilter(gray, 11, 17, 17)
cv2.imshow('Gray',image)
cv2.waitKey(0)

blurred=cv2.GaussianBlur(gray,(5,5),0)
cv2.imshow('Blurred ',blurred)
cv2.waitKey(0)

edges=cv2.Canny(blurred,30,150)
cv2.imshow("Canny ",edges)
cv2.waitKey(0)

#cv2.destroyAllWindows()

_,contours,_=cv2.findContours(edges.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)

contours=sorted(contours, key = x_cord_contour, reverse = False)
#contours= sorted(contours, key = cv2.contourArea, reverse = False)
full_number=[]


for c in contours:
    (x,y,w,h)=cv2.boundingRect(c)
    
    cv2.drawContours(image,contours,-1,(0,255,0),3)
    cv2.imshow("Contours ",image)
    
    if w>=5 and h>=25:
        roi=blurred[y:y+h,x:x+w]
        ret,roi=cv2.threshold(roi,127,255,cv2.THRESH_BINARY_INV)
        squared=makeSquare(roi)
        #cv2.imshow("asdfgh",squared)
        final=resize_to_pixels(20,squared)
        cv2.imshow("Final",final)
        final_array=final.reshape((1,400))
        final_array=final_array.astype(np.float32)
        ret,result,neighbours,dist=knn.findNearest(final_array,k=10)
        number=str(int(float(result[0])))
        full_number.append(number)
        cv2.rectangle(image,(x,y),(x+w,y+h),(0,0,255),2)
        cv2.putText(image,number,(x,x+155),cv2.FONT_HERSHEY_COMPLEX,2,(255,0,0),2)
        cv2.imshow("Image",image)
        cv2.waitKey(0)
        
        
cv2.destroyAllWindows()
print("The number is : "+''.join(full_number))

The number is : 80362


In [26]:
k = np.arange(10)
print(k)

[0 1 2 3 4 5 6 7 8 9]
